# Motivation

There are various analysis we can conduct on the same dataset. It is important to set the agenda before looking at the data to ensure we do not get lost along the way.

For this analysis, we are interested in understanding the key factors that affects the `resale price`.

# Retrieve data

We retrieve data from Data.gov.sg.

We can manually download the data, here we implemented a custom object to download data with a given URL.

In [ ]:
###############
# import data #
###############
import numpy as np  # numeric operations
import matplotlib.pyplot as plt  # for plotting
import pandas as pd  # dataframe
import pickle  # read stored location data

import gevent.monkey
gevent.monkey.patch_all()

from hdb_resale_data import (
    # retrive location data
    Location,
    # custom Data object
    Data,
)

In [ ]:
url = "https://data.gov.sg/dataset/7a339d20-3c57-4b11-a695-9348adfd7614/download"
data = Data(url)  # Data takes in a url: string

data.download(filename="../data/data.zip")  # Data stores the downloaded object with the given filename
data.zip_filename(zip_file="../data/data.zip")  # display the file names we downloaded

In [ ]:
# read the file inside the zip file
df = data.read_zip(zip_file="../data/data.zip", 
                   filename="resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv")

# Data profiling

We want to check for the following profiles:

1. Missing values
2. Data types

We will be keeping a profile of the list of data cleaning steps required before conducting data cleaning.

## Data preview

Sometimes the easiest way to identify data cleaning steps is to look at the data itself.

In [ ]:
df.info()

In [ ]:
df.head()

Observation:

1. `month` is coded as object (string) and not datetime
2. `remaining_lease` is coded as object (string) and not datetime
3. Location data is in string, hard to have any meaningful interpretation
4. `storey_range` is in string, hard to have any meaningful interpretation
5. `flat_type` is in string, hard to have any meaningful interpretation

Let's fix the issues before proceeding.

In [ ]:
##########################
# fix datetime for month #
##########################
# problem: month is in yyyy-mm, common datetime format require a day as well
# solution: concatenate a string '-01' before converting to datetime
# in order to ensure pandas convert our datetime object correctly,
# we will explictly input the format
df["month"] = pd.to_datetime(df["month"] + "-01", format="%Y-%m-%d")

In [ ]:
####################################
# fix datetime for remaining_lease #
####################################
# problem: remaining_lease is in years and month (string), we want a standardised unit
# solution: convert remaining_lease to years (year = month/12)
# we use Regex to extract out the years and month
years = df["remaining_lease"].str.extract("(\d+) years").astype("float")
months = df["remaining_lease"].str.extract("years (\d+) [months]|[month]").astype("float")

In [ ]:
# we have some missing months, let's make sure those are entries without a month data
df.loc[months.isna().values, "remaining_lease"].unique()

In [ ]:
# fill missing months values with 0
months.fillna(0, inplace=True)

In [ ]:
# add a new column with remaining lease in years
# and remove the old column
df["remaining_lease_years"] = years + months/12

In [ ]:
#####################
# fix location data #
#####################
# problem: location data is in string, it is hard to compare against different locations
# solution: retrive geolocation
# retriving geolocation is a more challenging task, we illustrate the idea here and
# execute it in a seperate script
loc = Location()
location = df["block"] + " " + df["street_name"]
links = loc.url1 + location + loc.url2
responses = loc.get_response(links.iloc[0])

In [ ]:
df["street_name"].str.replace("ST.", "ST", regex=False)

In [ ]:
# illustrate an example here
responses.json()

We will complete the location request in our data cleaning step as it is time consuming.

run `ETL.py` to repeat the data preprocessing steps

In [ ]:
with open("../data/response", "rb") as f:
    responses = pickle.load(f)

In [ ]:
# we have every row of data
len(responses), df.shape

In [ ]:
def get_location(response_obj):
    address = []
    postal = []
    latitude = []
    longtitude = []
    for content in iter(response_obj):
        try:
            results = content["results"]
            # take only the first result
            result = results[0]
            address.append(result["ADDRESS"])
            postal.append(result["POSTAL"])
            latitude.append(result["LATITUDE"])
            longtitude.append(result["LONGTITUDE"])
        except:
            address.append(None)
            postal.append(None)
            latitude.append(None)
            longtitude.append(None)
    return address, postal, latitude, longtitude

In [ ]:
address, postal, latitude, longtitude = get_location(responses)

In [ ]:
df["address"] = address
df["postal"] = postal
df["latitude"] = latitude
df["longtitude"] = longtitude

In [ ]:
# let's check if address are input correctly
# we observe that only ST. GEORGE'S is not able to retrive any data
# from our detailed analysis, we realise it's because of the full stop (.)
# most likely REST API interpret it as a reserved keyword
df.loc[df["address"].isna(), "street_name"].unique()

In [ ]:
# corrected ST. to ST
edited_subset_links = links[df["address"].isna()].replace("ST\.", "ST", regex=True)

In [ ]:
# there could be some network issue resulting in address failed to fetch
# we will iterate the replacement for a pre-defined number of tries
max_try = 10

In [ ]:
while df["address"].isna().sum() > 0 and max_try > 0:
    print(f"==========try {max_try}============")
    edited_subset_links = links[df["address"].isna()].replace("ST\.", "ST", regex=True)
    subset_response = loc.get_gresponse(edited_subset_links)
    subset_address = []
    for response in iter(subset_response):
        if response:
            subset_address.append(response.json())
        else:
            subset_address.append(None)
    subset_address, subset_postal, subset_latitude, subset_longtitude = get_location(subset_address)
    df.loc[df["address"].isna(), "address"] = subset_address
    df.loc[df["postal"].isna(), "postal"] = subset_postal
    df.loc[df["latitude"].isna(), "latitude"] = subset_latitude
    df.loc[df["longtitude"].isna(), "longtitude"] = subset_longtitude
    
    max_try -= 1 # decrease try by 1

In [ ]:
df.loc[df["address"].isna()]

In [ ]:
####################
# fix storey_range #
####################
# problem: storey_range is in string, it is hard to have meaning comparision
# solution: since the storey range is a numeric variable, let's take the first storey
df["storey_min"] = df["storey_range"].str.extract("(\d+) TO")
df["storey_min"] = df["storey_min"].astype("int")

## Data cleaning

As we have illustrated with the Data Preview section, simply by looking at the data and understanding
the data provides with us much things to do for data cleaning.

Now, let's remove the columns that is no longer needed.

In [ ]:
df.pop("storey_range");
df.pop("remaining_lease");

Now, notice although we have recoded the address but we did not remove the relevant columns.

We could technically remove those columns, but I have a feeling it might come in useful in the future.

Let's keep the columns for now, but have in mind that those columns are repeated.

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.isna().sum(axis=0)

In [ ]:
# we will save the data for our exploration next
df.to_csv("../data/hdb.csv")